# Parte 2 -  Features Categóricas

# Objetivos:

Na segunda etapa, o objetivo é analisar os dados do case e estruturar uma Feature Engineering básica apenas com os dados categóricos existentes, também sem transformar ou combinar features ou mesmo adicionar informações externas. 

Ao final do desafio, será treinado um modelo de regressão linear com as features obtidas. Esse modelo será testado contra uma massa de teste, separada previamente.

Também será agregada a feature engineering numérica em um segundo experimento de treino e avaliação, para comparar com a feat. eng. categórica sozinha. 


# Setup do Ambiente

## Magic Functions do Jupyter

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

## Imports de Libs Externas (padrão)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import numpy as np
import os
import pandas as pd

In [ ]:
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.externals import joblib

## Imports de Libs Locais

In [ ]:
from dataset import load_california_housing_prices
from pipeline import CategoricalFeaturesImputer, CategoricalToDummyFeaturesTransform, FeaturesChoiceTransform

## Carregando o Dataset:

In [ ]:
dataset = load_california_housing_prices()
x_train = dataset["train"]["x"]
y_train = dataset["train"]["y"]
x_test = dataset["test"]["x"]
y_test = dataset["test"]["y"]

# Feature Engineering c/ Features Categóricas

Features Categóricas são um pouco mais interessantes de tratar do que as numéricas, já que existem muitas maneiras de se transformar textos ou símbolos em valores numéricos. 

Vale lembrar que todo modelo de machine learning compreende o mundo através de valores numéricos, por serem modelos matemáticos de busca de solução ótima. Alguns frameworks atuais permitam que se coloquem valores simbólicos ou de texto marcados com a tag 'category' diretamente no dataset, mas por trás o proprio framework transforma esses dados em números.

## Análise da Distribuição das Categorias

### Verificação da quantidade de dados em cada categoria

É interessante verificar a quantidade de dados em cada categoria, pois categoriass mal representadas podem criar conceitos enviesados do modelo sobre o domínio. Por exemplo, em um dataset em que uma categoria só ocorra uma única vez e a variável dependente exatamente nesse elemento seja muito alta, um modelo treinado pode assumir que a presença dessa categoria já indique uma saída alta.

A seguir, é verificada a distribuiçlão das categorias na massa de treino.

In [ ]:
x_train["ocean_proximity"].fillna(" - NaN - ").value_counts()

Nem sempre é possível verificar a distribuição exata dos dados de produção, mas a massa de teste normalmente dá uma boa aproximação dela. A seguir é verificada a distribuição das categorias na massa de teste. 

In [ ]:
x_test["ocean_proximity"].fillna(" - NaN - ").value_counts()

#### Tratamento de categorias pouco representativos:

Pode-se observar que a categoria `ISLAND` tem uma representatividade mínima em todo o dataset, tornando essa categoria a única candidata à eliminação. Como já existem elementos com a categoria nula nesse dataset, a melhor estratégia é juntar a categoria `ISLAND` aos nulos e tratá-los (próxima seção).

###  Detecção e Tratamento de Nulos

Como já foram identificados elementos em que a categoria é nula, é importante tratar esses elementos apropriadamente.

Existem algumas estratégias para tratamento de nulos:

- Criar uma categoria `NULL` e usar como um símbolo válido do sistema;
- Criar modelos para inferir os valores a partir das outras features;
- Imputar um valor referente à distribuição: a `moda` (valor com a maior frequência)

Como existe a informação de que a variável categórica foi criada a partir da anotação manual do autor do dataset e que o mesmo utilizou as coordenadas `latitude` e `longitude`, a melhor estratégia é criar um `Imputer` que **busque a categoria do elemento usando as coordenadas geográfica**. 

--------------
#### Tarefa (2.1) 

Completar a implementação do transformador de dados `CategoricalFeaturesImputer`. 

A classe está no arquivo `pipeline.py`.

---------------

Dada a amostra dos dados de teste em que não há a informação categórica, mostrada a seguir

In [ ]:
x_valid = x_test[x_test.ocean_proximity.isnull()]
x_valid.head(10)

O código abaixo testa o `imputer` em elementos da amostra:

In [ ]:
valid_categories = ["<1H OCEAN", "INLAND", "NEAR OCEAN", "NEAR BAY"]
imputer = CategoricalFeaturesImputer(valid_categories).fit(x_train)
imputer.transform(x_valid).head(10)

##  Transformação em Dummy Features

A transformação em Dummy Features é a técnica em que os labels são formatados em dados categóricos consumíveis pelo modelo. O formato mais comum é usar uma representação em que cada label é uma nova feature binária onde o valor é **um** onde a feature é igual ao label e **zero** em todo o resto. 

Por exemplo, a transformação do vetor `[a, b, d, b, e, a, c]` seria da forma:

| a | b | c | d | e |
|---|---|---|---|---|
| 1 | 0 | 0 | 0 | 0 |
| 0 | 1 | 0 | 0 | 0 |
| 0 | 0 | 0 | 1 | 0 |
| 0 | 1 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 1 |
| 1 | 0 | 0 | 0 | 0 |
| 0 | 0 | 1 | 0 | 0 |

Em modelos lineares existe uma regra de ouro que uma das classes deve permanecer como 'Base' para não haver uma feature linearmente dependente dentro do dataset. Alguns modelos tratam esse problema internamente, mas ainda assim é uma boa prática a ser seguida.

Como é uma etapa de pré-processamento, essa transformação também deve ser feita como uma Feature Transformer.

--------------
#### Tarefa (2.2) 

Completar a implementação do transformador de dados `CategoricalToDummyFeaturesTransform`. 

A classe está no arquivo `pipeline.py`.

---------------

Para testar o novo transformador de dados, serão contabilizadas apenas as categorias `INLAND`, `NEAR OCEAN` e `NEAR BAY` na amostra:

In [ ]:
categories = ["INLAND", "NEAR OCEAN", "NEAR BAY"]

CategoricalToDummyFeaturesTransform(categories).transform(x_train).iloc[:10, -6:]

## Treinamento e Avaliação de um Modelo Linear

Continuando o treinamento do notebook anterior, serão feitas dois sets de treinamento e avaliação:

1. Apenas com a Feature Engineering Categórica;
2. Agregando as Feature Engineerings Numérica e Categórica.

###  Reload das massas de Treino e de Teste

As massas de dados de Treino e de Teste serão carregadas novamente para que seja aplicado o pipeline de pré-processamento em ambos desde o princípio. 

In [ ]:
dataset = load_california_housing_prices()
x_train = dataset["train"]["x"]
y_train = dataset["train"]["y"]
x_test = dataset["test"]["x"]
y_test = dataset["test"]["y"]

Deve-se remover os outliers da massa de treino usando a função construída para isso.

In [ ]:
keep_index = joblib.load(os.path.join("pipelines", "keep_index.pkl"))
x_train = x_train[keep_index]
y_train = y_train[keep_index]

###  Pipeline contendo apenas a Feature Engineering Categórica


####  Pipeline de Pré-Processamento

Apenas as etapas da Feature Engineering Categórica devem estar aqui.

In [ ]:
categories = ["<1H OCEAN", "INLAND", "NEAR OCEAN", "NEAR BAY"]

In [ ]:
cat_feat_eng_pipeline = Pipeline([
    ("categorical_imputer",      CategoricalFeaturesImputer(categories)),
    ("dummy_category_transform", CategoricalToDummyFeaturesTransform(categories)),
])

Esse pipeline será treinado e salvo para ser usado nos próximos notebook. 

In [ ]:
cat_feat_eng_pipeline.fit(x_train)
joblib.dump(
    value=cat_feat_eng_pipeline.fit(x_train),
    filename=os.path.join("pipelines", "categorical_feat_eng.pkl")
)

####  Pipeline Completo

Todas as etapas de pré-processamento devem estar incluídas nesse pipeline.

In [ ]:
chosen_features = [f"ocean_proximity: {c}" for c in categories]

In [ ]:
pipeline = Pipeline([
    ("categorical_feat_eng",  cat_feat_eng_pipeline),
    ("features_choice",       FeaturesChoiceTransform(chosen_features)),
    ("zscore",                StandardScaler()),
    ("predictor",             ElasticNet()),
])

####  Treinar e avaliar o modelo

In [ ]:
pipeline.fit(x_train, y_train)

Avaliação do modelo nas massas de **treino** e de **teste**.

In [ ]:
y_true = y_train
y_pred = pipeline.predict(x_train)
mse_tr = mean_squared_error(y_true=y_true, y_pred=y_pred)
r2_tr = r2_score(y_true=y_true, y_pred=y_pred)

In [ ]:
y_true = y_test
y_pred = pipeline.predict(x_test)
mse_te = mean_squared_error(y_true=y_true, y_pred=y_pred)
r2_te = r2_score(y_true=y_true, y_pred=y_pred)

In [ ]:
pd.DataFrame(
    index=["train", "test"],
    columns=["MSE", "R^2"],
    data=[
        [mse_tr, r2_tr],
        [mse_te, r2_te]
    ]
)

###  Pipeline contendo as Feature Engineerings Numérica & Categórica


####  Pipeline de Pré-Processamento

Todas as etapas de pré-processamento devem estar incluídas nesse pipeline.

In [ ]:
numerical_features = [
    "longitude", "latitude", 
    "housing_median_age", 
    "total_rooms", "total_bedrooms",  
    "population", "households", "median_income"
]

log_transform_features = [
    "total_rooms", "total_bedrooms", 
    "population", "households", 
    "median_income"
]

In [ ]:
chosen_features = (numerical_features +
                   [f"log_of_{c}" for c in log_transform_features] + 
                   [f"ocean_proximity: {c}" for c in categories])

In [ ]:
num_feat_eng_pipeline = joblib.load(os.path.join("pipelines", "numerical_feat_eng.pkl"))

In [ ]:
pipeline = Pipeline([
    ("numerical_feat_eng",    num_feat_eng_pipeline),
    ("categorical_feat_eng",  cat_feat_eng_pipeline),
    ("features_choice",       FeaturesChoiceTransform(chosen_features)),
    ("zscore",                StandardScaler()),
    ("predictor",             ElasticNet()),
])

####  Treinar e avaliar o modelo

In [ ]:
pipeline.fit(x_train, y_train)

Avaliação do modelo nas massas de **treino** e de **teste**.

In [ ]:
y_true = y_train
y_pred = pipeline.predict(x_train)
mse_tr = mean_squared_error(y_true=y_true, y_pred=y_pred)
r2_tr = r2_score(y_true=y_true, y_pred=y_pred)

In [ ]:
y_true = y_test
y_pred = pipeline.predict(x_test)
mse_te = mean_squared_error(y_true=y_true, y_pred=y_pred)
r2_te = r2_score(y_true=y_true, y_pred=y_pred)

In [ ]:
pd.DataFrame(
    index=["train", "test"],
    columns=["MSE", "R^2"],
    data=[
        [mse_tr, r2_tr],
        [mse_te, r2_te]
    ]
)